# Predicting Ames Housing Sales - Cleanup

#### Jump To:
- [Reading & Assessing our Dataset](#Asses)
- [Correlation Comparison to Nulls](#Corr_Null)
- [Handling Null Values](#Null_Values)
    - [Create Boolean](#bool)
    - [Fill it with 'NA', 0](#fill_na)
    - [Mean, Median or Other Column Value](#mmov)
    - [Drop](#drop)

### Imports 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<a id="Asses"></a>
### Reading in our datasets

In [2]:
# reading in our dataset
df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')
df.head() # make sure it is read in correctly

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [3]:
# info gives me column names, and dtypes all in one view, also a brief idea of null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [4]:
# checking for duplicates
df[df.duplicated()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


In [5]:
df.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


In [6]:
test_df.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
count,878.000000,8.780000e+02,878.000000,718.000000,878.000000,878.000000,878.000000,878.000000,878.000000,877.000000,...,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000
mean,1444.749431,7.162867e+08,58.268793,69.545961,10307.033030,6.050114,5.566059,1970.492027,1984.417995,106.236032,...,470.362187,93.430524,47.070615,24.064920,2.597950,14.830296,1.884966,48.498861,6.202733,2007.825740
std,850.838263,1.889103e+08,42.235407,23.533945,10002.674602,1.369065,1.128903,30.395895,20.450725,187.158908,...,212.734075,121.181702,68.180939,73.249463,24.962482,53.003794,29.916672,550.169317,2.642498,1.327861
min,2.000000,5.263021e+08,20.000000,21.000000,1477.000000,2.000000,1.000000,1880.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,692.250000,5.284841e+08,20.000000,59.000000,7297.250000,5.000000,5.000000,1954.000000,1967.000000,0.000000,...,322.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,1433.000000,5.354542e+08,50.000000,68.000000,9446.000000,6.000000,5.000000,1972.000000,1992.000000,0.000000,...,473.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2197.500000,9.071904e+08,70.000000,80.000000,11589.000000,7.000000,6.000000,1999.750000,2003.000000,172.000000,...,576.000000,170.750000,69.750000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2928.000000,1.007100e+09,190.000000,182.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,...,1488.000000,690.000000,742.000000,1012.000000,360.000000,576.000000,555.000000,15500.000000,12.000000,2010.000000


<a id="Corr_Null"></a>
#### Checking Correlation of variables to SalesPrice 
- Created a new table to show correlation and the null values in that column to help me pick features

In [7]:
corr = df.corr()[['SalePrice']].sort_values('SalePrice', ascending=False)

In [8]:
# I want to see only the columns that have null values
nulls = df.isnull().sum()
print(nulls[nulls > 0].sort_values(ascending=False))

Pool QC           2042
Misc Feature      1986
Alley             1911
Fence             1651
Fireplace Qu      1000
Lot Frontage       330
Garage Yr Blt      114
Garage Cond        114
Garage Qual        114
Garage Finish      114
Garage Type        113
Bsmt Exposure       58
BsmtFin Type 2      56
Bsmt Cond           55
Bsmt Qual           55
BsmtFin Type 1      55
Mas Vnr Type        22
Mas Vnr Area        22
Bsmt Half Bath       2
Bsmt Full Bath       2
Garage Cars          1
BsmtFin SF 1         1
BsmtFin SF 2         1
Bsmt Unf SF          1
Garage Area          1
Total Bsmt SF        1
dtype: int64


In [9]:
test_nulls = test_df.isnull().sum()
print(test_nulls[test_nulls > 0].sort_values(ascending=False))

Pool QC           874
Misc Feature      837
Alley             820
Fence             706
Fireplace Qu      422
Lot Frontage      160
Garage Yr Blt      45
Garage Finish      45
Garage Qual        45
Garage Cond        45
Garage Type        44
BsmtFin Type 2     25
BsmtFin Type 1     25
Bsmt Exposure      25
Bsmt Cond          25
Bsmt Qual          25
Mas Vnr Area        1
Mas Vnr Type        1
Electrical          1
dtype: int64


In [10]:
# Creating a table of correlation and NullValues, 
corr_null = corr.copy()
corr_null['NullVal'] = nulls
corr_null

,SalePrice,NullVal
SalePrice,1.000000,0
Overall Qual,0.800207,0
Gr Liv Area,0.697038,0
Garage Area,0.650270,1
Garage Cars,0.648220,1
Total Bsmt SF,0.628925,1
1st Flr SF,0.618486,0
Year Built,0.571849,0
Year Remod/Add,0.550370,0
Full Bath,0.537969,0


<a id="Null Values"></a>
### Assesing and Handling Null Values

<a id="bool"></a>
##### Home Features
- We will create a new column change the value of these column to boolean, indicating if there is a pool/fence/fireplace or not.
    - The original columns will have their values filled with NA lower down


In [11]:
# This function will create a new column of boolean value based on original column's nullness
def fill_bool(new_column, where_column):
    df[new_column] = np.where(df[where_column].isnull(), 0 , 1).astype('int64')
    test_df[new_column] = np.where(test_df[where_column].isnull(), 0 , 1).astype('int64') 

In [12]:
fill_bool('Has Pool', 'Pool QC')
fill_bool('Has Fence', 'Fence')
fill_bool('Has Fireplace', 'Fireplace Qu')
fill_bool('Alley', 'Alley') # this one I am going to overwrite 
fill_bool('Finished Basement', 'BsmtFin Type 1')

In [13]:
print(df['Alley'].value_counts(dropna=False)) # As a visual on how the new column's value show up

0    1911
1     140
Name: Alley, dtype: int64


<a id="fill_na"></a>
#### Filling the Nulls
- These columns will be filled with NA or 0 as the data documentation mentions that it is indicating that there are None of those items in that home

In [14]:
# create a function that will fill my Null columns with the 'NA' in both the train and test csvs
def fill_NA(column, val):
    df[column].fillna(val, inplace=True)
    test_df[column].fillna(val, inplace=True)

In [15]:
# data documentation indicates that the homes do not have these features

# I had to change the NA word to NotApp. instead because saving to csv would make NA null again
fill_NA('Pool QC', 'NotApp.') 
fill_NA('Misc Feature', 'NotApp.')
# Garage Values
fill_NA('Garage Type', 'NotApp.')
fill_NA('Garage Finish', 'NotApp.')
fill_NA('Garage Qual', 'NotApp.')
fill_NA('Garage Cond', 'NotApp.')
# Basement
fill_NA('Bsmt Qual', 'NotApp.')
fill_NA('Bsmt Cond', 'NotApp.')
fill_NA('Bsmt Exposure', 'NotApp.')
fill_NA('BsmtFin Type 1', 'NotApp.')
fill_NA('BsmtFin Type 2', 'NotApp.')
fill_NA('BsmtFin SF 1', 0)
fill_NA('BsmtFin SF 2', 0)
fill_NA('Bsmt Unf SF', 0)
fill_NA('Bsmt Full Bath', 0)
fill_NA('Bsmt Half Bath', 0)
# Masonry
fill_NA('Mas Vnr Type', 'NotApp.')
fill_NA('Mas Vnr Area', 0)

In [16]:
# Let's see what's left
print(df.isnull().sum().sort_values(ascending=False).head(10))
print('')
print(test_df.isnull().sum().sort_values(ascending=False).head(10))

Fence            1651
Fireplace Qu     1000
Lot Frontage      330
Garage Yr Blt     114
Total Bsmt SF       1
Garage Area         1
Garage Cars         1
Bsmt Exposure       0
Bsmt Cond           0
BsmtFin SF 2        0
dtype: int64

Fence                706
Fireplace Qu         422
Lot Frontage         160
Garage Yr Blt         45
Electrical             1
Finished Basement      0
Exter Qual             0
Roof Style             0
Roof Matl              0
Exterior 1st           0
dtype: int64


<a id="mmov"></a>
#### Mean, Median or Other Column Values
- These columns will either be filled with the mean, median or the same value as another column

In [17]:
# Generally the basement is the same size as 1st floor
df['Total Bsmt SF'].fillna(df['1st Flr SF'], inplace =True)
test_df['Total Bsmt SF'].fillna(test_df['1st Flr SF'], inplace =True)

In [18]:
df['Total Bsmt SF'].isnull().sum()

0

#### Lot Frontage


In [19]:
# All property has part of the street attached to the from part of the home, I will fill these with the mean
df['Lot Frontage'].fillna((df['Lot Frontage'].mean().round(0)), inplace=True)
test_df['Lot Frontage'].fillna((test_df['Lot Frontage'].mean().round(0)), inplace=True)

#### Garage

In [20]:
# Since it is only 1 value missing, I will just fill it with the median (both in the same row)
df['Garage Area'].fillna(df['Garage Area'].median(), inplace=True)
df['Garage Cars'].fillna(df['Garage Cars'].median(), inplace=True)
# test df has no null for Garage Area

#### Electrical in Test

In [21]:
# Looking at the data documentation there is one more value that isn't listed here 'Mix' I was going to assume that the one missing
# value is Mix, however since I plan to use One Hot later on, creating one more category means one more column, so I will put this
# one missing value in with the majority
print(test_df['Electrical'].value_counts())
test_df['Electrical'].fillna('SBrkr', inplace=True)

SBrkr    813
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64


<a id="drop"></a>
### Dropping Columns
- These Columns will be dropped, they were either turned into 'dummy variables' or not needed


In [22]:
def drop_col(column):
    df.drop(columns= column, inplace=True)
    test_df.drop(columns= column, inplace=True)        

In [23]:
drop_col('Fireplace Qu') # this is fireplace quantity, since there are alot of missing values, and no way to determine how many
                        # fireplaces a home has, it will be handled by our boolean column we made earlier

drop_col('Garage Yr Blt') # Most homes have their Garage's built at end of construction so maybe not in the same year the house
                        # was built but pretty close by.
    
drop_col('Fence') # It has been dummied    


In [24]:
print('null values left on Train', df.isnull().sum().sum())
print('null values left on Test', test_df.isnull().sum().sum())

null values left on Train 0
null values left on Test 0


### Save Changes to new csv

In [25]:
df.to_csv('./datasets/train_cleanup.csv', index = False)

In [26]:
test_df.to_csv('./datasets/test_cleanup.csv', index = False)